In [34]:
from StopGoClassifier import StopGoClassifier
import pandas as pd
import geopandas as gpd

classifier = StopGoClassifier()


In [35]:
gps_data = pd.read_excel("PositionsUnit-LongLat.xlsx", index_col = [0])


In [36]:
#Convert Longs and Lats to floats. 

def cnvrtCoord(val: str) -> float:
    sgn = +1
    if val[0] == 'S' or val[0] == 'W':
        sgn = -1
    return float(val[1:]) * sgn  

gps_data['long'] = [cnvrtCoord(x) for x in gps_data['long'].tolist()]
gps_data['lat'] = [cnvrtCoord(x) for x in gps_data['lat'].tolist()]  


In [37]:
gps_data['DateTimeOfPosition'] = pd.to_datetime(gps_data['DateTimeOfPosition'],  utc=True)


In [38]:
d = gps_data.set_index('DateTimeOfPosition').tz_convert('Europe/Berlin').reset_index()


In [41]:
geometry = gpd.points_from_xy(gps_data.lat, gps_data.long)
gdf = gpd.GeoDataFrame(gps_data, geometry=geometry)
gdf = gdf.set_crs('EPSG:4326') # interpret raw GPS records as WGS84 / EPSG 4326
gdf = gdf.to_crs('EPSG:3035') # project sample into the ETRS89-extended / LAEA Europe space


In [44]:
# extract x / y values from geopandas geometry object
gdf['x'] = list(map(lambda x: x.x, gdf.geometry.values))
gdf['y'] = list(map(lambda x: x.y, gdf.geometry.values))

In [47]:
gps_data = gdf[['DateTimeOfPosition', 'x', 'y', 'long', 'lat']].copy()


In [49]:
gps_data

,DateTimeOfPosition,x,y,long,lat
0,2023-05-01 04:29:18.987000+00:00,3.237269e+06,4.004805e+06,52.154859,5.376457
1,2023-05-01 04:28:58.437000+00:00,3.237270e+06,4.004809e+06,52.154865,5.376504
2,2023-05-01 04:26:58.420000+00:00,3.237692e+06,4.005066e+06,52.158796,5.379862
3,2023-05-01 04:24:58.953000+00:00,3.236942e+06,4.005590e+06,52.152371,5.388205
4,2023-05-01 04:22:58.207000+00:00,3.237477e+06,4.006749e+06,52.157829,5.404613
...,...,...,...,...,...
995,2023-03-28 08:22:19.030000+00:00,3.239464e+06,4.011236e+06,52.178164,5.468254
996,2023-03-28 08:22:18.600000+00:00,3.239464e+06,4.011236e+06,52.178164,5.468254
997,2023-03-28 08:22:18.600000+00:00,3.239342e+06,4.010895e+06,52.176882,5.463401
998,2023-03-28 08:22:18.600000+00:00,3.239546e+06,4.010620e+06,52.178559,5.459197


In [57]:
classifier.read(gps_data.DateTimeOfPosition, gps_data.y, gps_data.x)

In [58]:
identified_stops_df = classifier.run()


In [59]:
identified_stops_df

id,start,stop,duration,x,y
0,2023-05-01 04:11:32.010000+00:00,2023-04-28 13:38:04.677000+00:00,-225207.333,NaN,NaN
1,2023-04-27 22:03:29.430000+00:00,2023-04-21 12:10:58.137000+00:00,-553951.293,NaN,NaN
2,2023-04-21 08:00:19.033000+00:00,2023-04-20 16:32:26.473000+00:00,-55672.560,NaN,NaN
3,2023-04-20 12:38:05.760000+00:00,2023-04-20 04:44:55.870000+00:00,-28389.890,NaN,NaN
4,2023-04-20 04:30:55.757000+00:00,2023-04-13 13:13:38.450000+00:00,-573437.307,NaN,NaN
5,2023-04-13 12:29:56.720000+00:00,2023-04-13 04:41:36.677000+00:00,-28100.043,NaN,NaN
6,2023-04-13 04:16:39.190000+00:00,2023-04-09 08:50:51.993000+00:00,-329147.197,NaN,NaN
7,2023-04-09 08:41:00.213000+00:00,2023-04-09 07:47:48.557000+00:00,-3191.656,NaN,NaN
8,2023-04-08 06:41:40.450000+00:00,2023-04-08 05:51:21.180000+00:00,-3019.270,NaN,NaN
9,2023-04-07 13:08:09.233000+00:00,2023-04-06 15:23:12.270000+00:00,-78296.963,NaN,NaN
